# FINAL TA: Multi Label
Name : linkv12 <br>
Date : 25 November 2022<br>
Finalize the experiment for single label <br>

# Setup

In [ ]:
%%time
import tensorflow as tf
from tensorflow.keras import layers, models, losses, Model, optimizers

In [4]:
# import os for stuff
import os
import re

import copy
import string
import numpy as np
import pandas as pd
import pickle
# for graphing
import matplotlib.pyplot as plt
from IPython.display import display


from datetime import datetime

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix, hamming_loss, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection  import KFold

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
# to supress invalid warning 
old_err_state = np.seterr(invalid ='ignore')

In [ ]:
# nltk
import nltk
nltk.download(['stopwords','punkt'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

porterStemmer = nltk.stem.PorterStemmer()

## Constant

In [6]:
BATCHSIZE = 8
EPOCH = 100
EMBEDDING_DIM = 750

SAVE_AE = False

sep = '\\' if os.name == 'nt' else '/'
cwd = os.getcwd()

FOLDERNAME = r'{}{}data'.format(cwd, sep, sep)
DATASETNAME = r'{}{}elsvier.csv'.format(FOLDERNAME, sep)

RESULTS = r"{}{}results{}".format(cwd, sep, sep)

COLUMN_NAME = ['abstract','subjareas']

In [ ]:
print(cwd)
print(FOLDERNAME)
print(DATASETNAME)
print(RESULTS)

## Dataset Wrangling
Make sure the data type and formating is close to a final expected type<br>

In [8]:
# make a single df 
df = pd.read_csv(DATASETNAME, usecols=COLUMN_NAME)


# shuffle the DataFrame rows
shuff = df.sample(frac = 1, random_state=4473, ignore_index=True)

shuff
# # label encoding
# # creating instance of labelencoder
# labelencoder = LabelEncoder()
# # Assigning numerical values and storing in another column
# shuff['label_cat'] = labelencoder.fit_transform(shuff['label'])

,abstract,subjareas
0,Models that estimate land-use impacts on biodi...,"['AGRI', 'ENVI']"
1,This paper investigates food waste dynamics in...,"['ECON', 'ENVI']"
2,Piwi-interacting RNAs (piRNAs) are important f...,['BIOC']
3,The demise of Miocene carbonate build-ups in t...,['EART']
4,"Over recent decades, Land Use and Cover Change...",['ENVI']
...,...,...
39520,"Background To date, there have been few studie...","['MEDI', 'PSYC']"
39521,We argue that a hexagonal grid with simple int...,"['COMP', 'MATH']"
39522,Experimental and theoretical studies show that...,['AGRI']
39523,This review discusses recent developments in t...,"['BIOC', 'PHAR']"


In [9]:
# c08 - 
# we want to change 
# wrangling subjareas 
# change it to list from str

def reg_ex(s:str) :
    # input : str  - "['MULT', 'SOCI']"
    # output: list - ['MULT', 'SOCI']
    
    inp = re.sub("[^a-zA-Z,]+", "", s)
    inp = inp.split(",")
    return inp

shuff['subjareas_list'] = list(map(reg_ex, shuff['subjareas'].tolist()))
shuff

,abstract,subjareas,subjareas_list
0,Models that estimate land-use impacts on biodi...,"['AGRI', 'ENVI']","[AGRI, ENVI]"
1,This paper investigates food waste dynamics in...,"['ECON', 'ENVI']","[ECON, ENVI]"
2,Piwi-interacting RNAs (piRNAs) are important f...,['BIOC'],[BIOC]
3,The demise of Miocene carbonate build-ups in t...,['EART'],[EART]
4,"Over recent decades, Land Use and Cover Change...",['ENVI'],[ENVI]
...,...,...,...
39520,"Background To date, there have been few studie...","['MEDI', 'PSYC']","[MEDI, PSYC]"
39521,We argue that a hexagonal grid with simple int...,"['COMP', 'MATH']","[COMP, MATH]"
39522,Experimental and theoretical studies show that...,['AGRI'],[AGRI]
39523,This review discusses recent developments in t...,"['BIOC', 'PHAR']","[BIOC, PHAR]"


In [10]:
# for sanity
mlb = MultiLabelBinarizer()
mlb.fit(shuff['subjareas_list'])

mlb.classes_

array(['AGRI', 'ARTS', 'BIOC', 'BUSI', 'CENG', 'CHEM', 'COMP', 'DECI',
       'DENT', 'EART', 'ECON', 'ENER', 'ENGI', 'ENVI', 'HEAL', 'IMMU',
       'MATE', 'MATH', 'MEDI', 'MULT', 'NEUR', 'NURS', 'PHAR', 'PHYS',
       'PSYC', 'SOCI', 'VETE'], dtype=object)

In [11]:
# for sanity
class_name_list = list(mlb.classes_)
class_dict = {}
class_num = 0

for class_name in class_name_list :
    lbl_num = np.argmax(mlb.transform([[class_name]])[0])
    class_dict[lbl_num] = class_name
    class_num += 1
    
print(class_num)
print(class_dict)

27
{0: 'AGRI', 1: 'ARTS', 2: 'BIOC', 3: 'BUSI', 4: 'CENG', 5: 'CHEM', 6: 'COMP', 7: 'DECI', 8: 'DENT', 9: 'EART', 10: 'ECON', 11: 'ENER', 12: 'ENGI', 13: 'ENVI', 14: 'HEAL', 15: 'IMMU', 16: 'MATE', 17: 'MATH', 18: 'MEDI', 19: 'MULT', 20: 'NEUR', 21: 'NURS', 22: 'PHAR', 23: 'PHYS', 24: 'PSYC', 25: 'SOCI', 26: 'VETE'}


In [12]:
# Reff CELLNUM  : 11
#      CELLNAME : currie
# c11 - making label_cat to dataframe

# desc: a workaround just like above c10 for sanity
#       its just a workaround
def mlb_transform(_list) :
    return mlb.transform([_list])[0]

shuff['label_cat'] = list(map(mlb_transform, shuff['subjareas_list']))
shuff

,abstract,subjareas,subjareas_list,label_cat
0,Models that estimate land-use impacts on biodi...,"['AGRI', 'ENVI']","[AGRI, ENVI]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,This paper investigates food waste dynamics in...,"['ECON', 'ENVI']","[ECON, ENVI]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."
2,Piwi-interacting RNAs (piRNAs) are important f...,['BIOC'],[BIOC],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The demise of Miocene carbonate build-ups in t...,['EART'],[EART],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
4,"Over recent decades, Land Use and Cover Change...",['ENVI'],[ENVI],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
...,...,...,...,...
39520,"Background To date, there have been few studie...","['MEDI', 'PSYC']","[MEDI, PSYC]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39521,We argue that a hexagonal grid with simple int...,"['COMP', 'MATH']","[COMP, MATH]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39522,Experimental and theoretical studies show that...,['AGRI'],[AGRI],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39523,This review discusses recent developments in t...,"['BIOC', 'PHAR']","[BIOC, PHAR]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Preprocessing

In [13]:
%%time
def custom_standardization(input_data, verbose= False):
    # print original string
    if verbose :
        print("original string: " ,input_data)
    
    # lowering
    lowercase = input_data.lower()
    
    # print lowercase string
    if verbose :
        print('lower string: ', lowercase)
    
    
    # clean the text
    stripped = re.sub(" #39;", "\'", lowercase)
    stripped = re.sub(" quot;", "\"", stripped)
    # remove stopword
    # regex magic : r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*'
    stripped = re.sub(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*',"", stripped)
    
    # print remove stopword string
    if verbose :
        print('Stopword removal: ', stripped)
    
    # strip punctuiation
    stripped = re.sub(f"[{re.escape(string.punctuation)}]", " ", stripped)
    # remove double space
    # two or more whitespace
    stripped = re.sub(r"(\s\s*)", " ", stripped)
    stripped = re.sub(r"(\s+$)", "", stripped)
    
    # tokenize
    tokens = word_tokenize(stripped)

    # print tokenize
    if verbose :
        print("toknize: ", tokens)
    
    # stemming
    temp_cont = [porterStemmer.stem(word) for word in tokens]
    
    # print stemmed
    if verbose:
        print("stemmed: ", temp_cont)
    
    porter_    = ' '.join(temp_cont)
    return porter_

CPU times: total: 0 ns
Wall time: 0 ns


In [ ]:
# For Testing
# t_list = ["The boys thought it was quite remarkable that the oxen never turned north throughout the days",
#          "The fir tree has a comforting smell as it sways in the wind",
#          "My house is ablaze because I forgot to turn off the oven",
#          "Grandma loves soft bread and tea, though she hates eating meat given her age",
#          "The three-legged goat running across the irradiated field"]
# # t_list
# # for report
# # t_list = copy.deepcopy(shuff['abstract'].tolist()[1000:1005])
# for t in t_list :
#     tp = custom_standardization(t, verbose=True)

In [14]:
%%time
# CELLNUM : 13
# CELLNAME: clown

c13_start = datetime.now()
# clean text data
clean_text = list(map(custom_standardization, shuff['abstract'].tolist()))
shuff['clean_text'] = clean_text

c13_duration = datetime.now() - c13_start
print("cell:map_cleaning-text \nExecution Time : {:.2f} s".format(c13_duration.total_seconds()))

cell:map_cleaning-text 
Execution Time : 211.26 s
CPU times: total: 3min 31s
Wall time: 3min 31s


In [15]:
shuff

,abstract,subjareas,subjareas_list,label_cat,clean_text
0,Models that estimate land-use impacts on biodi...,"['AGRI', 'ENVI']","[AGRI, ENVI]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",model estim land use impact biodivers multipl ...
1,This paper investigates food waste dynamics in...,"['ECON', 'ENVI']","[ECON, ENVI]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",paper investig food wast dynam retail altern f...
2,Piwi-interacting RNAs (piRNAs) are important f...,['BIOC'],[BIOC],"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",piwi interact rna pirna import genom regul acr...
3,The demise of Miocene carbonate build-ups in t...,['EART'],[EART],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",demis miocen carbon build up brows basin north...
4,"Over recent decades, Land Use and Cover Change...",['ENVI'],[ENVI],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",recent decad land use cover chang lucc trend m...
...,...,...,...,...,...
39520,"Background To date, there have been few studie...","['MEDI', 'PSYC']","[MEDI, PSYC]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",background date studi dmdd examin risk factor ...
39521,We argue that a hexagonal grid with simple int...,"['COMP', 'MATH']","[COMP, MATH]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",argu hexagon grid simpl intermedi node robust ...
39522,Experimental and theoretical studies show that...,['AGRI'],[AGRI],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",experiment theoret studi show mortal impos pop...
39523,This review discusses recent developments in t...,"['BIOC', 'PHAR']","[BIOC, PHAR]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",review discuss recent develop use non code rna...


## Build TF-IDF SKlearn

In [16]:
%%time
# CELLNUM  : 15
# CELLNAME : zoan

c15_start = datetime.now()
# generate & fit
vectorizer = TfidfVectorizer(min_df=0.01, 
                             max_df = 0.8, 
                             ngram_range=(1,2))
vectorizer.fit(shuff['clean_text'])
dim_size = vectorizer.get_feature_names_out().shape[0]
print("Vector Size : ", dim_size)
c15_duration = datetime.now() - c15_start
print("cell:train_tf-idf \nExecution Time : {0:.2f} s".format(c15_duration.total_seconds()))

Vector Size :  1784
cell:train_tf-idf 
Execution Time : 13.46 s
CPU times: total: 13.4 s
Wall time: 13.5 s


## K-Fold

In [17]:
K5_fold = KFold(n_splits=5)
fold = []
# fold -> [[train_idex, val_index, test_index], ...,[train_idex, val_index, test_index]]

for train_index, test_index in K5_fold.split(shuff['clean_text']):
    val_index = test_index[int(len(test_index) / 2): ]
    test_index = test_index[:int(len(test_index)/2)]
    fold.append([train_index, val_index, test_index])

# ANN

## Build NN
Build a NN from predetermined composition <br>
Need `input_size` as a parameter          <br>


In [18]:
%time

def build_nn(input_size:int):
    input_layer = layers.Input(shape=(input_size,))

    hdn  = layers.Dense(900, activation='selu') (input_layer)
    hdn = layers.BatchNormalization(momentum=0.99, epsilon=0.0001)(hdn)
    hdn  = layers.Dense(600, activation='elu') (hdn)
    hdn = layers.BatchNormalization(momentum=0.99, epsilon=0.0001)(hdn)
    hdn  = layers.Dense(900, activation='selu') (hdn)
    hdn  = layers.Dense(700, activation='selu') (hdn)

    # using logits so this is how the output look
    predictions = layers.Dense(27, activation='sigmoid', name="predictions")(hdn)
    
    model = tf.keras.Model(input_layer, predictions)
    loss=tf.keras.losses.BinaryCrossentropy()
    # Compile the model with binary crossentropy loss and an adam optimizer.
    model.compile(loss= loss,
              optimizer="adam", 
              metrics=[tf.keras.metrics.BinaryAccuracy()])
    
    return model

CPU times: total: 0 ns
Wall time: 0 ns


## Metrics for ANN

In [19]:
# function to classification_report homebrew
def _report_classification(confusion_matrix, class_name= class_dict, as_df=True) :
    # class_name -> a dict with int as key and str as value
    
    kf_tbl = {}
    for i in range(confusion_matrix.shape[0]) :
        c_name = class_name[i]
        elm_conf_matrix = confusion_matrix[i]
        kf_tbl[c_name] = {}
        # TP 
        kf_tbl[c_name]["TP"] = elm_conf_matrix[1,1]
        # FN
        # baris koma belakang
        kf_tbl[c_name]["FN"] = elm_conf_matrix[1,0]
        # FP
        # klo liat kolom koma depan
        kf_tbl[c_name]["FP"] = elm_conf_matrix[0,1]
        # FN
        kf_tbl[c_name]["TN"] = elm_conf_matrix[0,0]
        # Precision
        kf_tbl[c_name]["Precision"] = np.nan_to_num(kf_tbl[c_name]["TP"]/(kf_tbl[c_name]["TP"] + kf_tbl[c_name]["FP"]))

        # Recall
        kf_tbl[c_name]["Recall"] = np.nan_to_num((kf_tbl[c_name]["TP"]/(kf_tbl[c_name]["TP"] + kf_tbl[c_name]["FN"])))
        # f1-score per-class
        _temp = (kf_tbl[c_name]["Precision"]* kf_tbl[c_name]["Recall"])
        _temp = _temp/(kf_tbl[c_name]["Precision"] + kf_tbl[c_name]["Recall"])
        _temp = np.nan_to_num(_temp)
        kf_tbl[c_name]["f1-score"] = np.nan_to_num(2*_temp)
        
    if as_df : 
        t_tbl = {"labels" : [],
                 "TP"     : [],
                 "FN"     : [],
                 "FP"     : [],
                 "TN"     : [],
                 "Precision" : [],
                 "Recall"    : [],
                 "f1-score"  : []
                }
        for key in list(kf_tbl.keys()) :
            t_tbl['labels'].append(key)
            t_tbl['TP'].append(kf_tbl[key]['TP'])
            t_tbl['FN'].append(kf_tbl[key]['FN'])
            t_tbl['FP'].append(kf_tbl[key]['FP'])
            t_tbl['TN'].append(kf_tbl[key]['TN'])
            t_tbl['Precision'].append(kf_tbl[key]['Precision'])
            t_tbl['Recall'].append(kf_tbl[key]['Recall'])
            t_tbl['f1-score'].append(kf_tbl[key]['f1-score'])
        df = pd.DataFrame.from_dict(t_tbl)
        df = df.set_index('labels')
        
        return df
    
    return kf_tbl

In [20]:

def get_f1(cls_report) :
    cls_report_type = type(cls_report)
    if cls_report_type is dict :
        cls_num = len(list(cls_report.keys()))
        cls_keys = list(cls_report.keys())
        
        # macro
        # get all f1-score then just average it to cls_num
        _f1_list = []
        for key in cls_keys :
            _f1_list.append(cls_report[key]['f1-score'])
        _macro_f1 = sum(_f1_list) / cls_num
        _f1_list = None
        
        # weigthed
        # support_proportion = support / sum(support)
        # sum(f1*support_proportion)
        _f1_list = []
        _sum_support = 0
        for key in cls_keys :
            _support = cls_report[key]['TP'] + cls_report[key]['FN']
            _f1_list.append(cls_report[key]['f1-score'] * _support)
            _sum_support += _support
        _weighted_f1 = sum(_f1_list) / _sum_support
        _f1_list = None
                        
        # micro
        # all_class_TP FP  amnd FN
        # using that just count f1 normally
        _sum_TP = 0
        _sum_FP = 0
        _sum_FN = 0
        for key in cls_keys :
            _sum_TP += cls_report[key]['TP']
            _sum_FP += cls_report[key]['FP']
            _sum_FN += cls_report[key]['FN']
        _micro_f1 = _sum_TP / (_sum_TP + (0.5*(_sum_FP + _sum_FN)))
                        
        rslt = {'f1' : {
                'micro'    : _micro_f1,
                'macro'    : _macro_f1,
                'weighted' : _weighted_f1}
              }
        return rslt
    elif cls_report_type is pd.core.frame.DataFrame :
        cls_num = cls_report.shape[0]
        
        # macro
        _macro_f1 = sum(cls_report['f1-score'].tolist()) / len(cls_report['f1-score'].tolist())
        
        # weighted
        # support = tp + fn
        _cls_TP = cls_report['TP'].tolist()
        _cls_FN = cls_report['FN'].tolist()
        _cls_f1 = cls_report['f1-score'].tolist()
        
        _support = [_cls_TP[i] + _cls_FN[i] for i in range(len(_cls_TP))]
        _sum_support = sum(_support)
        
        # sum(f1[i] * (support[i] / sum(support)))
        _weighted_f1 = sum([_cls_f1[i] * (_support[i] / _sum_support) for i in range(len(_cls_f1))])
        
        # micro
        _sum_TP = sum(cls_report['TP'].tolist())
        _sum_FP = sum(cls_report['FP'].tolist())
        _sum_FN = sum(cls_report['FN'].tolist())
        _micro_f1 = _sum_TP / (_sum_TP + (0.5*(_sum_FP + _sum_FN)))
        
        rslt = {'f1' : {
                'micro'    : _micro_f1,
                'macro'    : _macro_f1,
                'weighted' : _weighted_f1}
              }
        return rslt
    else :
        err = {'f1' : {
                'micro'    : -4473,
                'macro'    : -4473,
                'weighted' : -4473}
              }
        
        return err
    

In [21]:
# LOCAL CONSTANT
# single most important

THRESHOLD = 0.5

def _to_multi_hot(float_pred) :
    # use map
    _pred_bool = float_pred > THRESHOLD
    return _pred_bool.astype(int)    

In [22]:
# temp save protocol
# save model to file 
# purge session 
# reload the model

def _blaze_p(ae, encoder) :
    t_ae_name = ae._name
    t_e_name = encoder._name
    
    f_ae_name = "{}.h5".format(t_ae_name)
    f_e_name = "{}.h5".format(t_e_name)
    
    ae.save(f_ae_name)
    encoder.save(f_e_name)
    
    
    # clear session, rem from memory
    tf.keras.backend.clear_session()
    
    ae = models.load_model(f_ae_name)
    encoder = models.load_model(f_e_name)
    
    
    return ae, encoder

In [23]:
def _average_dataframe(list_dframe: list) :
    for i in range(len(list_dframe)) :
        if i == 0 :
            f_avg = list_dframe[i]
        else :
            f_avg = f_avg.add(list_dframe[i])

    # divide by amount of fold
    f_avg = f_avg.div(len(list_dframe))

    # round down
    f_avg['TP'] = f_avg['TP'].astype('int')
    f_avg['FN'] = f_avg['FN'].astype('int')
    f_avg['FP'] = f_avg['FP'].astype('int')
    f_avg['TN'] = f_avg['TN'].astype('int')
    
    # return an average of all dataframe
    return f_avg

# Evaluation

Evaluate the f1-measure and acc

### Function to Evaluate Vanilla Performance

In [24]:
def _eval_control_multi(save_file=False) :
    # i  - counter
    i = 1
    
    # t_res -> f1-macro foreach fold
    # t_loss-> loss foreach fold
    # t_acc -> acc foreach fold
    t_res = []
    t_loss = []
    t_acc =[]
    
    t_hamming_loss = []
    t_acc_score = []
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        mf_cstart = datetime.now()
        
        # clear session _del model from memory
        tf.keras.backend.clear_session()
        
        # prepare dataset
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()
        
        
        y_train = np.array(shuff['label_cat'][train_index].tolist()).ravel()
        y_train = np.reshape(y_train, (int(y_train.shape[0]/class_num), class_num))
        
        y_val = np.array(shuff['label_cat'][val_index].tolist()).ravel()
        y_val = np.reshape(y_val, (int(y_val.shape[0]/class_num), class_num))

        y_test = np.array(shuff['label_cat'][test_index].tolist()).ravel()
        y_test = np.reshape(y_test, (int(y_test.shape[0]/class_num), class_num))
        
        
        # build nn
        nn_name = "multi_control_{}".format(i)
        nn_model = build_nn(input_size=dim_size)
        nn_model._name = nn_name
        # print(nn_model._name)
        
        # train - fit
        fit_history = nn_model.fit(X_train, 
                                     y_train, 
                                     epochs=100, 
                                     batch_size=75, 
                                     validation_data = (X_val, y_val),
                                     verbose=0)
        

        # test
        # save confusion matrix too
        print("Evaluate on test data ", i)
        results = nn_model.evaluate(X_test, y_test, batch_size=128, verbose=0)
        print("test loss, test acc:", results)
        pred = nn_model.predict(X_test, verbose=0)
        pred = list(map(_to_multi_hot, pred))
        

        result_df = _report_classification(multilabel_confusion_matrix(y_test, pred))
        result_metric = get_f1(result_df)
        
        
        list_dataframe.append(result_df)
        t_res.append(result_metric['f1']['macro'])
        t_loss.append(results[0])
        t_acc.append(results[1])
        # get hamming loss and accuracy score
        t_hamming_loss.append(hamming_loss(y_test, pred))
        t_acc_score.append(accuracy_score(y_test, pred))
        print('hamming loss:' ,t_hamming_loss[-1])
        print('accuracy score:' ,t_acc_score[-1])
        print('f1-macro:', t_res[-1])
        mf_cdur = datetime.now() - mf_cstart
        print("func:eval_model on fold {} \nExecution Time : {:.2f} s".format(i , mf_cdur.total_seconds()))
        print()
        
        
        
        i += 1
    
    d_result = {'f1-macro' : sum(t_res) / len(t_res),
                'loss' : sum(t_loss) / len(t_loss),
                'bin_acc'  : sum(t_acc) / len(t_acc), 
                'hamming_loss' : sum(t_hamming_loss) / len(t_hamming_loss),
                'accuracy_score' : sum(t_acc_score) / len(t_acc_score),
                'avg_conf' : _average_dataframe(list_dataframe)
               }
    
    return d_result

### Control Experiment Result

In [25]:
control_measurement = _eval_control_multi(save_file=True)

Evaluate on test data  1
test loss, test acc: [0.8149200081825256, 0.9445756673812866]
hamming loss: 0.055424351476983054
accuracy score: 0.2866902834008097
f1-macro: 0.48424994526696785
func:eval_model on fold 1 
Execution Time : 430.58 s

Evaluate on test data  2
test loss, test acc: [0.7216058969497681, 0.9443038702011108]
hamming loss: 0.05569613135402609
accuracy score: 0.2932692307692308
f1-macro: 0.47169829973937555
func:eval_model on fold 2 
Execution Time : 416.27 s

Evaluate on test data  3
test loss, test acc: [0.7562237977981567, 0.9433104395866394]
hamming loss: 0.05668953366321788
accuracy score: 0.2909919028340081
f1-macro: 0.47495172576999484
func:eval_model on fold 3 
Execution Time : 419.82 s

Evaluate on test data  4
test loss, test acc: [0.7983012795448303, 0.9448005557060242]
hamming loss: 0.0551994301994302
accuracy score: 0.28846153846153844
f1-macro: 0.4821293447299008
func:eval_model on fold 4 
Execution Time : 421.16 s

Evaluate on test data  5
test loss, test

In [27]:
print(control_measurement)
print("AVERAGE PRECISION : ", control_measurement['avg_conf']['Precision'].mean())
print("AVERAGE RECALL : ", control_measurement['avg_conf']['Recall'].mean())

{'f1-macro': 0.47627092937022725, 'loss': 0.7707617521286011, 'bin_acc': 0.944150173664093, 'hamming_loss': 0.05584982756035388, 'accuracy_score': 0.2896761133603239, 'avg_conf':          TP   FN   FP    TN  Precision    Recall  f1-score
labels                                                    
AGRI    194  212  139  3406   0.590075  0.477668  0.524926
ARTS     25   74   31  3820   0.440246  0.248895  0.314058
BIOC    492  335  230  2894   0.684136  0.594202  0.633876
BUSI     31   64   49  3807   0.397421  0.325931  0.353876
CENG     66  141   73  3670   0.474820  0.322316  0.382505
CHEM    106  161  111  3573   0.487187  0.396107  0.434863
COMP    121  171  107  3551   0.540172  0.414569  0.463915
DECI      9   45   22  3875   0.304303  0.166893  0.212175
DENT      0    4    0  3947   0.000000  0.000000  0.000000
EART    175  102   79  3594   0.688861  0.631721  0.659036
ECON     38   72   24  3816   0.631852  0.345094  0.439079
ENER    150  133   98  3570   0.604917  0.532089  0.56

### Function to Compare with other alg

### SVM Result

In [28]:
def _eval_svm(save_file=False) :
    i = 1
    t_res = []
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        # print(train_index, val_index, test_index)
        
        ## ERROE
        # train_index = train_index[:5]
        # val_index = val_index[:5]
        # test_index = test_index[:5]
        
        
        f_svmstart = datetime.now()
        
        
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()
        
        print("x train:",X_train.shape)
        print("x val  :",X_val.shape)
        print("x test :",X_test.shape)
        # print(type(X_train))
        
        # print(X_val)
        # i dont know
        y_train = np.array(shuff['label_cat'][train_index].tolist()).ravel()
        y_train = np.reshape(y_train, (int(y_train.shape[0]/class_num), class_num))
        
        y_val = np.array(shuff['label_cat'][val_index].tolist()).ravel()
        y_val = np.reshape(y_val, (int(y_val.shape[0]/class_num), class_num))

        y_test = np.array(shuff['label_cat'][test_index].tolist()).ravel()
        y_test = np.reshape(y_test, (int(y_test.shape[0]/class_num), class_num))
        
        print("y train:",y_train.shape)
        print("y val  :",y_val.shape)
        print("y test :",y_test.shape)
        
        print("unique train:", np.unique(y_train, axis=0).shape[0])
        print("unique val  :", np.unique(y_val, axis=0).shape[0])
        print("unique test :", np.unique(y_test, axis=0).shape[0])
        
        # print(y_val)
        # print(y_train[:5])
        
        t_time = datetime.now()
        print("prepare training fold-{} .... took a while".format(i))
        
        
        clf = MultiOutputClassifier(svm.SVC(kernel='linear', class_weight ='balanced',decision_function_shape='ovr')).fit(X_train, y_train)
        t_dur = datetime.now() - t_time  
        print('elapsed time for training : {:2f} s'.format(t_dur.total_seconds()))
        print('fold-{} ...done training'.format(i))
        pred = clf.predict(X_test)
        
        result_df = _report_classification(multilabel_confusion_matrix(y_test, pred))
        
        print(result_df)
        print(get_f1(result_df))
        list_dataframe.append(result_df)
        # delete
        X_train, X_val,X_test = None, None, None
        y_train, y_val,y_test = None, None, None
        clf = None
        
        print('finish fold-{}'.format(i))
        
        i += 1
    # average all fold
    result_avg = _average_dataframe(list_dataframe)
    result_f1 = get_f1(result_avg)

    f_svmduration = datetime.now() - f_svmstart
    print("func:evaluate_svm \nExecution Time : {:2f} s".format(f_svmduration.total_seconds()))
    return {'result' : result_avg,
            'f1'     : result_f1}
    
# x = _eval_svm()

In [29]:
# x = _eval_svm()
svm_result = _eval_svm(save_file=False)
print(svm_result)

print("AVERAGE PRECISION : ", svm_result['result']['Precision'].mean())
print("AVERAGE RECALL : ", svm_result['result']['Recall'].mean())

x train: (31620, 1784)
x val  : (3953, 1784)
x test : (3952, 1784)
y train: (31620, 27)
y val  : (3953, 27)
y test : (3952, 27)
unique train: 332
unique val  : 269
unique test : 259
prepare training fold-1 .... took a while
elapsed time for training : 7730.191265 s
fold-1 ...done training
         TP   FN   FP    TN  Precision    Recall  f1-score
labels                                                    
AGRI    324  113  489  3026   0.398524  0.741419  0.518400
ARTS     61   37  297  3557   0.170391  0.622449  0.267544
BIOC    664  173  486  2629   0.577391  0.793309  0.668344
BUSI     61   27  275  3589   0.181548  0.693182  0.287736
CENG    161   38  573  3180   0.219346  0.809045  0.345123
CHEM    245   56  563  3088   0.303218  0.813953  0.441839
COMP    242   62  548  3100   0.306329  0.796053  0.442413
DECI     21   26  152  3753   0.121387  0.446809  0.190909
DENT      0    8    0  3944   0.000000  0.000000  0.000000
EART    241   42  336  3333   0.417678  0.851590  0.560465
EC

In [30]:
# Naive Bayes

def _eval_nb(save_file=False) :
    f_nbstart = datetime.now()
    i = 1
    t_res = []
    list_dataframe = []
    for train_index, val_index, test_index in fold :
        f_fold_nb = datetime.now()
        
        
        X_train = vectorizer.transform(shuff['clean_text'][train_index]).toarray()
        X_val = vectorizer.transform(shuff['clean_text'][val_index]).toarray()
        X_test = vectorizer.transform(shuff['clean_text'][test_index]).toarray()


        y_train = np.array(shuff['label_cat'][train_index].tolist()).ravel()
        y_train = np.reshape(y_train, (int(y_train.shape[0]/class_num), class_num))
        
        y_val = np.array(shuff['label_cat'][val_index].tolist()).ravel()
        y_val = np.reshape(y_val, (int(y_val.shape[0]/class_num), class_num))

        y_test = np.array(shuff['label_cat'][test_index].tolist()).ravel()
        y_test = np.reshape(y_test, (int(y_test.shape[0]/class_num), class_num))
        
        t_time = datetime.now()
        print("prepare training fold-{} .... took a while".format(i))
        mb_nb = MultinomialNB(alpha = 0.6)
        clf = MultiOutputClassifier(mb_nb).fit(X_train, y_train)
        t_dur = datetime.now() - t_time  
        print('elapsed time for training : {:2f} s'.format(t_dur.total_seconds()))
    
        pred = clf.predict(X_test)
        
        result_df = _report_classification(multilabel_confusion_matrix(y_test, pred))
        
        print(result_df)
        print(get_f1(result_df))
        list_dataframe.append(result_df)
        # delete
        X_train, X_val,X_test = None, None, None
        y_train, y_val,y_test = None, None, None
        
        print('finish fold-{}'.format(i))
        i += 1
    # average all fold
    result_avg = _average_dataframe(list_dataframe)
    result_f1 = get_f1(result_avg)

    f_nbduration = datetime.now() - f_nbstart
    print("func:evaluate_naivebayes \nExecution Time : {:2f} s".format(f_nbduration.total_seconds()))
    return {'result' : result_avg,
            'f1'     : result_f1}

In [31]:
nb_result = _eval_nb()

prepare training fold-1 .... took a while
elapsed time for training : 3.328988 s
         TP   FN   FP    TN  Precision    Recall  f1-score
labels                                                    
AGRI     94  343   48  3467   0.661972  0.215103  0.324698
ARTS      2   96    0  3854   1.000000  0.020408  0.040000
BIOC    461  376  228  2887   0.669086  0.550777  0.604194
BUSI      5   83   14  3850   0.263158  0.056818  0.093458
CENG     36  163   21  3732   0.631579  0.180905  0.281250
CHEM     61  240   71  3580   0.462121  0.202658  0.281755
COMP     88  216   35  3613   0.715447  0.289474  0.412178
DECI      0   47    0  3905   0.000000  0.000000  0.000000
DENT      0    8    0  3944   0.000000  0.000000  0.000000
EART    137  146   41  3628   0.769663  0.484099  0.594360
ECON     12   99   14  3827   0.461538  0.108108  0.175182
ENER     94  192   37  3629   0.717557  0.328671  0.450839
ENGI    283  299  189  3181   0.599576  0.486254  0.537002
ENVI    315  310  176  3151   0.64

In [32]:
print(nb_result)

{'result':          TP   FN   FP    TN  Precision    Recall  f1-score
labels                                                    
AGRI     98  308   48  3497   0.670511  0.242676  0.355998
ARTS      1   98    0  3851   0.450000  0.009778  0.019113
BIOC    446  381  252  2871   0.638784  0.539416  0.584810
BUSI      6   88   20  3836   0.246057  0.070299  0.108939
CENG     29  177   26  3717   0.530173  0.143565  0.225439
CHEM     60  206   77  3606   0.439833  0.228364  0.299898
COMP     83  209   36  3623   0.697623  0.282981  0.402280
DECI      0   54    0  3897   0.000000  0.000000  0.000000
DENT      0    4    0  3947   0.000000  0.000000  0.000000
EART    126  151   30  3643   0.804549  0.453839  0.579711
ECON      9  101   15  3826   0.390905  0.084770  0.138320
ENER    101  182   39  3629   0.717860  0.358851  0.477603
ENGI    272  318  204  3157   0.571461  0.460786  0.510107
ENVI    317  308  158  3166   0.666876  0.507493  0.576243
HEAL      0   79    0  3872   0.000000  0.000

In [33]:
print("AVERAGE PRECISION : ", nb_result['result']['Precision'].mean())
print("AVERAGE RECALL : ", nb_result['result']['Recall'].mean())

AVERAGE PRECISION :  0.4976196525964238
AVERAGE RECALL :  0.2900077932455214


## Performance Analysis

### Training History

#### Vanilla Training History

In [ ]:
# print dict keys, what does history contain
# print(vanilla_measurement['history'][0].history.keys())
# dont print figure automaticly
_i = plt.ioff()
for i in range(len(vanilla_measurement['history'])) :
    # print(vanilla_measurement['history'][i].history.keys())
    fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(10,3))
    ax1.plot(vanilla_measurement['history'][i].history['loss'], label="loss")
    ax1.plot(vanilla_measurement['history'][i].history['val_loss'], label="val_loss")
    ax1.legend(fontsize='x-small')
    # Add a footnote below and to the right side of the chart
    ax1.annotate('*lower is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)

    ax2.plot(vanilla_measurement['history'][i].history['binary_accuracy'], label="binary_accuracy")
    ax2.plot(vanilla_measurement['history'][i].history['val_binary_accuracy'], label="val_binary_accuracy")
    # ax2.plot(vanilla_measurement['history'][i].history['val_binary_accuracy'], label="val_binary_accuracy")
    ax2.legend(fontsize='x-small')
    
    ax2.set_yticks(np.arange(0,1.2,0.1))
    ax2.set_xlim(left=-5, right=104)
    
    # max accuracy
    ax2.plot(np.arange(-10,110,1), np.ones(120), color='red')
    # Add a footnote below and to the right side of the chart
    ax2.annotate('*higher is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)

    # fig.suptitle("Fold {} : Vanilla Performance".format(i+1))
    fig.suptitle("Fold {} : Baseline Performance".format(i+1))
    f_name = "{}img{}multi_base_perf_f{}.png".format(RESULTS, sep, i+1)
    fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)
    display(fig)

# close all fig after printed
plt.close('all')
_i = plt.ion()

#### Encoded Training History

In [ ]:
# dont print figure automaticly
_i = plt.ioff()
for i in range(len(encoded_measurement['history'])) :
    # print(vanilla_measurement['history'][i].history.keys())
    fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(10,3))
    ax1.plot(encoded_measurement['history'][i].history['loss'], label="loss")
    ax1.plot(encoded_measurement['history'][i].history['val_loss'], label="val_loss")
    ax1.legend(fontsize='x-small')
    # Add a footnote below and to the right side of the chart
    ax1.annotate('*lower is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)

    ax2.plot(encoded_measurement['history'][i].history['binary_accuracy'], label="binary_accuracy")
    ax2.plot(encoded_measurement['history'][i].history['val_binary_accuracy'], label="val_binary_accuracy")
    ax2.legend(fontsize='x-small')
    
    ax2.set_yticks(np.arange(0,1.2,0.1))
    ax2.set_xlim(left=-5, right=104)
    
    # max accuracy
    ax2.plot(np.arange(-10,110,1), np.ones(120), color='red')
    # Add a footnote below and to the right side of the chart
    ax2.annotate('*higher is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)

    fig.suptitle("Fold {} : Encoded Performance".format(i+1))
    f_name = "{}img{}multi_encoded_perf_f{}.png".format(RESULTS, sep, i+1)
    fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)
    display(fig)

# close all fig after printed
plt.close('all')
_i = plt.ion()

#### Training History Comparison

In [ ]:
# dont print figure automaticly
_i = plt.ioff()
for i in range(len(encoded_measurement['history'])) :
    fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(10,3))
    
    ax1.plot(vanilla_measurement['history'][i].history['val_loss'], label="base loss")
    ax1.plot(encoded_measurement['history'][i].history['val_loss'], label="encoded loss")

    # ax1.plot(vanilla_measurement['history'][i].history['val_loss'], label="vanilla loss")
    ax1.legend(fontsize='x-small')
    # Add a footnote below and to the right side of the chart
    ax1.annotate('*lower is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)

    ax2.plot(vanilla_measurement['history'][i].history['val_binary_accuracy'], label="base_binary accuracy")
    ax2.plot(encoded_measurement['history'][i].history['val_binary_accuracy'], label="encoded_binary accuracy")

    # ax2.plot(vanilla_measurement['history'][i].history['val_binary_accuracy'], label="vanila_binary accuracy")
    ax2.legend(fontsize='x-small')
    
    ax2.set_yticks(np.arange(0,1.2,0.1))
    ax2.set_xlim(left=-5, right=104)
    
    # max accuracy
    ax2.plot(np.arange(-10,110,1), np.ones(120), color='red')
    # Add a footnote below and to the right side of the chart
    ax2.annotate('*higher is better',
                xy = (1.0, -0.15),
                xycoords='axes fraction',
                ha='right',
                va="center",
                fontsize=7)
    

    fig.suptitle("Fold {} : Validation Performance Comparisson".format(i+1))
    f_name = "{}img{}multi_val_comp_f{}.png".format(RESULTS, sep, i+1)
    fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)
    # fig.tight_layout()
    display(fig)

# close all fig after printed
plt.close('all')
_i = plt.ion()

## Metrics

### f1-micro comparison

In [ ]:
# comaprison on f1-micro
vanila_micro = []
encoded_micro = []

vanila_macro = []
encoded_macro = []

vanila_weight = []
encoded_weight = []
for i in range(5) :
    vanila_micro.append(vanilla_measurement['metric'][i]['f1']['micro'])
    encoded_micro.append(encoded_measurement['metric'][i]['f1']['micro'])


_i = plt.ioff()

# set width of bar
barWidth = 0.25
fig, ax1 = plt.subplots(1, 1, figsize=(10,3))

# Set position of bar on X axis
br1 = np.arange(len(vanila_micro))
br2 = [x + barWidth for x in br1]

# Make the plot
# ax1.bar(br1, vanila_micro, width = barWidth,
#         edgecolor ='grey', label ='vanila')
ax1.bar(br1, vanila_micro, width = barWidth,
        edgecolor ='grey', label ='base')
ax1.bar(br2, encoded_micro, width = barWidth,
        edgecolor ='grey', label ='encoded')


ax1.legend(fontsize='x-small')

plt.ylabel('f1-score', fontsize = 11)
plt.xticks([r + barWidth for r in range(len(vanila_micro))],
        ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'])
fig.suptitle("f1-micro comparison")
# fig.tight_layout()
display(fig)

f_name = "{}img{}multi_f1_micro_comp.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)

# close all fig after printed
plt.close('all')
_i = plt.ion()
    

### f1-macro comparison

In [ ]:
# comaprison on f1-macro


vanila_macro = []
encoded_macro = []

for i in range(5) :
    vanila_macro.append(vanilla_measurement['metric'][i]['f1']['macro'])
    encoded_macro.append(encoded_measurement['metric'][i]['f1']['macro'])

_i = plt.ioff()

# set width of bar
barWidth = 0.25
fig, ax1 = plt.subplots(1, 1, figsize=(10,3))

# Set position of bar on X axis
br1 = np.arange(len(vanila_macro))
br2 = [x + barWidth for x in br1]

# Make the plot
# ax1.bar(br1, vanila_macro, width = barWidth,
#         edgecolor ='grey', label ='vanila')
ax1.bar(br1, vanila_macro, width = barWidth,
        edgecolor ='grey', label ='base')
ax1.bar(br2, encoded_macro, width = barWidth,
        edgecolor ='grey', label ='encoded')


ax1.legend(fontsize='x-small')

plt.ylabel('f1-score', fontsize = 11)
plt.xticks([r + barWidth for r in range(len(vanila_micro))],
        ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'])
fig.suptitle("f1-macro comparison")
# fig.tight_layout()
display(fig)

f_name = "{}img{}multi_f1_macro_comp.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)

# close all fig after printed
plt.close('all')
_i = plt.ion()
    

### f1-weighted comparison

In [ ]:
# comaprison on f1-weighted

vanila_weight = []
encoded_weight = []
for i in range(5) :
    vanila_weight.append(vanilla_measurement['metric'][i]['f1']['weighted'])
    encoded_weight.append(encoded_measurement['metric'][i]['f1']['weighted'])

_i = plt.ioff()

# set width of bar
barWidth = 0.25
fig, ax1 = plt.subplots(1, 1, figsize=(10,3))

# Set position of bar on X axis
br1 = np.arange(len(vanila_weight))
br2 = [x + barWidth for x in br1]

# Make the plot
# ax1.bar(br1, vanila_weight, width = barWidth,
#         edgecolor ='grey', label ='vanila')
ax1.bar(br1, vanila_weight, width = barWidth,
        edgecolor ='grey', label ='base')
ax1.bar(br2, encoded_weight, width = barWidth,
        edgecolor ='grey', label ='encoded')


ax1.legend(fontsize='x-small')

plt.ylabel('f1-score', fontsize = 11)
plt.xticks([r + barWidth for r in range(len(vanila_weight))],
        ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'])
fig.suptitle("f1-weighted comparison")
# fig.tight_layout()
display(fig)

f_name = "{}img{}multi_f1_weight_comp.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)

# close all fig after printed
plt.close('all')
_i = plt.ion()

### Hamming Loss

In [ ]:
# comparison on hamming loss
# add foot note lower is better

vanila_hloss = []
encoded_hloss= []
for i in range(5) :
    vanila_hloss.append(vanilla_measurement['metric'][i]['hamming_loss'])
    encoded_hloss.append(encoded_measurement['metric'][i]['hamming_loss'])

_i = plt.ioff()

# set width of bar
barWidth = 0.25
fig, ax1 = plt.subplots(1, 1, figsize=(10,3))

# Set position of bar on X axis
br1 = np.arange(len(vanila_hloss))
br2 = [x + barWidth for x in br1]

# Make the plot
# ax1.bar(br1, vanila_hloss, width = barWidth,
#         edgecolor ='grey', label ='vanila')
ax1.bar(br1, vanila_hloss, width = barWidth,
        edgecolor ='grey', label ='base')
ax1.bar(br2, encoded_hloss, width = barWidth,
        edgecolor ='grey', label ='encoded')


ax1.legend(fontsize='x-small')

plt.ylabel('hamming loss', fontsize = 11)
plt.xticks([r + barWidth for r in range(len(vanila_weight))],
        ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'])
fig.suptitle("hamming loss comparison")
# fig.tight_layout()
display(fig)
f_name = "{}img{}multi_hamm_loss_comp.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)

# close all fig after printed
plt.close('all')
_i = plt.ion()

### Accuracy Score

In [ ]:
# comaprison on accuracy_score
# its using sklearn.metrics.accuracy_score
# add foot note higher is better

vanila_ascore = []
encoded_ascore= []
for i in range(5) :
    vanila_ascore.append(vanilla_measurement['metric'][i]['accuracy_score'])
    encoded_ascore.append(encoded_measurement['metric'][i]['accuracy_score'])

_i = plt.ioff()

# set width of bar
barWidth = 0.25
fig, ax1 = plt.subplots(1, 1, figsize=(10,3))

# Set position of bar on X axis
br1 = np.arange(len(vanila_ascore))
br2 = [x + barWidth for x in br1]

# Make the plot
# ax1.bar(br1, vanila_ascore, width = barWidth,
#         edgecolor ='grey', label ='vanila')
ax1.bar(br1, vanila_ascore, width = barWidth,
        edgecolor ='grey', label ='base')
ax1.bar(br2, encoded_ascore, width = barWidth,
        edgecolor ='grey', label ='encoded')


ax1.legend(fontsize='x-small')

plt.ylabel('accuracy score', fontsize = 11)
plt.xticks([r + barWidth for r in range(len(vanila_ascore))],
        ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5'])
fig.suptitle("accuracy score comparison")
# fig.tight_layout()
display(fig)

f_name = "{}img{}multi_acc_score_comp.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)
# close all fig after printed
plt.close('all')
_i = plt.ion()

# Show Important Table

Output a latex, average it for all fold.

## Vanilla

### Average

In [ ]:
f_avg = None

# sum all dataframe
for i in range(len(vanilla_measurement['result'])) :
    if i == 0 :
        f_avg = vanilla_measurement['result'][i]
    else :
        f_avg = f_avg.add(vanilla_measurement['result'][i])

# divide by amount of fold
f_avg = f_avg.div(len(vanilla_measurement['result']))

# round down
f_avg['TP'] = f_avg['TP'].astype('int')
f_avg['FN'] = f_avg['FN'].astype('int')
f_avg['FP'] = f_avg['FP'].astype('int')
f_avg['TN'] = f_avg['TN'].astype('int')

print(f_avg.style.to_latex())

### Fold

Convert fold to latex table

In [ ]:
# c40
# loop through all dataframe
for i in range(len(vanilla_measurement['result'])) :
    print("Vanilla Fold-{} Tabel".format(i+1))
    print(vanilla_measurement['result'][i].style.to_latex())
    # print(vanilla_measurement['result'][i], '\n')
    

### F1 All fold

In [ ]:
# loop through all f1
v_micro = []
v_macro = []
v_weighted = []

# v_micro = ['micro']
# v_macro = ['macro']
# v_weighted = ['weighted']

v_colName = []
for i in range(len(vanilla_measurement['metric'])) :
    v_micro.append(vanilla_measurement['metric'][i]['f1']['micro'])
    v_macro.append(vanilla_measurement['metric'][i]['f1']['macro'])
    v_weighted.append(vanilla_measurement['metric'][i]['f1']['weighted'])
    v_colName.append('fold-{}'.format(i+1))

v_micro.append(sum(v_micro)/len(v_micro))
v_macro.append(sum(v_macro)/len(v_macro))
v_weighted.append(sum(v_weighted)/len(v_weighted))
v_colName.append('average')
v_f1 = pd.DataFrame({'micro' : v_micro, 'macro' : v_macro, 'weighted' : v_weighted}).T
v_f1.columns = v_colName
v_f1.index.name='f1'
print(v_f1)

print(v_f1.style.to_latex())

## Encoded Average

### Average

In [ ]:
f_avg = None

# sum all dataframe
for i in range(len(encoded_measurement['result'])) :
    if i == 0 :
        f_avg = encoded_measurement['result'][i]
    else :
        f_avg = f_avg.add(encoded_measurement['result'][i])

# divide by amount of fold
f_avg = f_avg.div(len(encoded_measurement['result']))

# round down
f_avg['TP'] = f_avg['TP'].astype('int')
f_avg['FN'] = f_avg['FN'].astype('int')
f_avg['FP'] = f_avg['FP'].astype('int')
f_avg['TN'] = f_avg['TN'].astype('int')

print(f_avg.style.to_latex())

### Fold

Convert fold to latex table

In [ ]:
# loop through all dataframe
for i in range(len(encoded_measurement['result'])) :
    print("Encoded Fold-{} Tabel".format(i+1))
    print(encoded_measurement['result'][i].style.to_latex())
    # print(encoded_measurement['result'][i], '\n')

    

### F1 All fold

In [ ]:
# loop through all f1
v_micro = []
v_macro = []
v_weighted = []

v_colName = []
for i in range(len(encoded_measurement['metric'])) :
    v_micro.append(encoded_measurement['metric'][i]['f1']['micro'])
    v_macro.append(encoded_measurement['metric'][i]['f1']['macro'])
    v_weighted.append(encoded_measurement['metric'][i]['f1']['weighted'])
    v_colName.append('fold-{}'.format(i+1))
    
v_micro.append(sum(v_micro)/len(v_micro))
v_macro.append(sum(v_macro)/len(v_macro))
v_weighted.append(sum(v_weighted)/len(v_weighted))
v_colName.append('average')

v_f1 = pd.DataFrame({'micro' : v_micro, 'macro' : v_macro, 'weighted' : v_weighted}).T
v_f1.columns = v_colName
v_f1.index.name='f1'

print('Encoded F1 All')
print(v_f1)

print(v_f1.style.to_latex())

## Analisis Dataset

In [ ]:
baris = shuff.shape[0]
print("Jumlah baris pada dataset : {}".format(baris))

In [ ]:
# Distribusi dan jumlah tiap kategori
# shuff['label'].tolist()
class_dict


lbl_ = np.array(shuff['label_cat'].tolist())
lbl_ = sum(lbl_, 0)
# lbl_ = ???
# so we sum it all the lsit to a single list contain count of all label with len 27

count_dict = {}
for key, item in class_dict.items() :
    count_dict[item] = (lbl_[key] / baris) * 100
    
count_dict

In [ ]:

_i = plt.ioff()

label = list(count_dict.keys())
count = list(count_dict.values())

fig, ax1 = plt.subplots(1, 1, figsize=(10,6))
 

ax1.barh(label, count, align='center')
ax1.set_yticks(label, labels=label)
ax1.invert_yaxis()  # labels read top-to-bottom
ax1.set_xlabel('Persentase Data')
ax1.set_title('Distribusi Label dalam Dataset')

plt.show()
f_name = "{}img{}multi_label_distribution.png".format(RESULTS, sep)
fig.savefig(f_name, bbox_inches ="tight",
                        pad_inches = 0.15)

# close all fig after printed
plt.close('all')
_i = plt.ion()